In [2]:
import sklearn
assert sklearn.__version__ >= "0.20"

import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >="2.0"

import numpy as np
import os

np.random.seed(42)
tf.random.set_seed(42)

import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
def plot_image(image):
    plt.imshow(image, cmap="gray", interpolation='nearest')
    plt.axis("off")
    
def plot_color_image(image):
    plt.imshow(image, interpolation='nearest')
    plt.axis("off")

# What is a Convolution?

In [ ]:
from sklearn.datasets import load_sample_image

china=load_sample_image("china.jpg")/255
flower=load_sample_image("flower.jpg")/255
images=np.array([china, flower])
batch_size, height, width, channels = images.shape

filters=np.zeros(shape=(7,7,channels, 2),dtype=np.float32)
filters[:,3,:,0] =1 # vertical line
filters[3,:,:,1] =1 # horizontal line

outputs = tf.nn.conv2d(images, filters, strides=1, padding="SAME")

plt.imshow(outputs[0,:,:,1], cmap='gray')
plt.axis('off')

In [ ]:
for image_index in (0,1):
    for feature_map_index in (0,1):
        plt.subplot(2,2, image_index*2 + feature_map_index+1)
        plot_image(outputs[image_index,:,:,feature_map_index])


In [ ]:
def crop(image):
    return images[150:220, 130:250]

In [ ]:
plot_image(crop(images[0,:,:,0]))

for feature_map_index, filename in enumerate(["china_vertical", "china_horizontal"]):
    plot_image(crop(outputs[0,:,:,feature_map_index]))
    

In [ ]:
plot_image(filters[:,:,0,0])
plot_image(filters[:,:,0,1])

## Convolutional Layer

In [ ]:
conv = keras.layers.Conv2D(filters =2, kernel_size=7, strides=1,
                          padding="SAME", activation='relu', input_shape=outputs.shape)

In [ ]:
conv_outputs=conv(images)
conv_outputs.shape

次元はバッチサイズ、高さ、幅、チャンネル。  
この畳込み層は2つのフィルターを持つのでチャンネルの次元数は2になる。

In [ ]:
plt.figure(figsize =(10,6))
for image_index in (0,1):
    for feature_map_index in (0,1):
        plt.subplot(2,2, image_index*2+feature_map_index+1 )
        plot_image(crop(conv_outputs[image_index, :,:,feature_map_index]))


In [ ]:
conv.set_weights([filters, np.zeros(2)])

In [ ]:
conv_outputs=conv(images)
conv_outputs.shape

In [ ]:
plt.figure(figsize =(10,6))
for image_index in (0,1):
    for feature_map_index in (0,1):
        plt.subplot(2,2, image_index*2+feature_map_index+1 )
        plot_image(crop(conv_outputs[image_index, :,:,feature_map_index]))


## VALID vs SAME padding

In [ ]:
def feature_map_size(input_size, kernel_size, strides=1, padding="SAME"):
    if padding=='SAME':
        return (input_size-1)//strides +1
    else:
        return (input_size -kernel_size)//strides+1 

In [ ]:
def pad_before_and_padded_size(input_size, kernel_size, strides=1):
    fmap_size=feature_map_size(input_size, kernel_size, strides)
    padded_size=max((fmap_size -1)*strides + kernel_size, input_size)
    pad_before=(padded_size-input_size)//2
    return pad_before, padded_size

In [ ]:
def manual_same_padding(images, kernel_size, strides=1):
    if kernel_size==1:
        return images.astype(np.float32)
    batch_size, height, width, channels=images.shape
    top_pad, padded_height=pad_before_and_padded_size(height, kernel_size, strides)
    left_pad, padded_widtht=pad_before_and_padded_size(width, kernel_size, strides)
    padded_shape  = [batch_size, padded_height, padded_width, channels]
    padded_images=np.zeros(padded_shape, dtype=np.float32)
    padded_images[:, top_pad:hieght+top_pad, left_pad:width+left_pad, :]=images
    return padded_images

In [ ]:
kernel_size =7
strides =2

conv_valid = keras.layers.Conv2D(filters=1, kernel_size=kernel_size, strides=strides, padding="VALID")
conv_same = keras.layers.Conv2D(filters=1, kernel_size=kernel_size, strides=strides, padding="SAME")

valid_output = conv_valid(manual_same_padding(images, kernel_size, strides))

conv_same.build(tf.TensorShape(images.shape))

conv_same.set_weights(conv_valid.get_weights())

same_output = conv_same(images.astype(np.float32))

assert np.allclose(valid_output.numpy(), same_output.numpy())

# Pooling layer

## Max pooling